# FIN411X – Final Portfolio Project
## ChatGPT + Google Colab + Python Code + CAPM Workflow

This notebook runs the full workflow for **one** selected portfolio
from a set of **30 predefined diversified portfolios (P01–P30)**.

**Instructions for students:**
1. Run the cells from top to bottom.
2. Change `student_id` and `portfolio_id` in the section marked clearly.
3. Download your summary CSV and upload it with your report.


In [ ]:
# Install required libraries (run this cell first)
!pip -q install yfinance quantstats statsmodels


In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import quantstats as qs

print("Libraries imported successfully.")


In [ ]:
# === Project settings (dates, budget, risk-free rate) ===
START_DATE = "2025-01-02"
END_DATE   = "2025-04-02"

BUDGET_AED  = 600_000
AED_PER_USD = 3.6725
BUDGET_USD  = BUDGET_AED / AED_PER_USD

RF_ANNUAL = 0.04
RF_DAILY  = (1 + RF_ANNUAL) ** (1/252) - 1

MARKET_TICKER = "^GSPC"

print("Budget (USD):", BUDGET_USD)
print("Daily risk-free rate:", RF_DAILY)


In [ ]:
# === Fixed weight pattern (same for every portfolio) ===
# Equity 1 = 18%, Equity 2 = 15%, Equity 3 = 12%, Equity 4 = 10%,
# Equity 5 = 10%, Equity 6 = 8%, ETF1 = 8%, ETF2 = 7%, Commodity = 6%, Crypto = 6%
WEIGHTS_PATTERN = np.array([0.18, 0.15, 0.12, 0.10, 0.10, 0.08, 0.08, 0.07, 0.06, 0.06])
print("Weights sum to:", WEIGHTS_PATTERN.sum())


In [ ]:
# === 30 pre-defined portfolios (tickers only) ===
PORTFOLIOS = {
    "P01": ["TSLA","AAPL","JNJ","JPM","CSCO","NVDA","QQQ","XLV","GLD","ADA-USD"],
    "P02": ["BAC","MSFT","AAPL","NVDA","GOOGL","TSLA","XLV","SPY","DBC","ETH-USD"],
    "P03": ["ORCL","BAC","JPM","NKE","JNJ","NFLX","SPY","IWM","DBA","SOL-USD"],
    "P04": ["JNJ","AMZN","GOOGL","DIS","TSLA","MSFT","XLF","QQQ","USO","SOL-USD"],
    "P05": ["V","JNJ","MSFT","NKE","TSLA","CSCO","XLF","QQQ","DBC","SOL-USD"],
    "P06": ["UNH","AMZN","NFLX","DIS","XOM","PFE","EEM","VNQ","USO","BTC-USD"],
    "P07": ["AMZN","V","TSLA","META","UNH","MSFT","VNQ","QQQ","SLV","BTC-USD"],
    "P08": ["NKE","DIS","XOM","JNJ","AMZN","META","XLK","XLE","GLD","ETH-USD"],
    "P09": ["KO","PFE","ORCL","CSCO","UNH","BAC","XLK","VNQ","DBA","SOL-USD"],
    "P10": ["GOOGL","NVDA","META","NKE","DIS","NFLX","SPY","XLV","GLD","ETH-USD"],
    "P11": ["PFE","BAC","XOM","V","NKE","KO","XLF","EFA","SLV","ADA-USD"],
    "P12": ["JNJ","TSLA","GOOGL","AMZN","NFLX","UNH","EFA","XLE","USO","BTC-USD"],
    "P13": ["AAPL","MSFT","META","NVDA","JPM","CSCO","QQQ","EEM","GLD","BTC-USD"],
    "P14": ["DIS","KO","ORCL","PFE","V","NFLX","XLV","IWM","DBA","BNB-USD"],
    "P15": ["BAC","JPM","JNJ","XOM","UNH","META","SPY","XLK","DBC","ETH-USD"],
    "P16": ["NKE","AAPL","MSFT","TSLA","AMZN","NVDA","QQQ","XLF","GLD","BTC-USD"],
    "P17": ["GOOGL","DIS","KO","ORCL","CSCO","PFE","VNQ","EEM","SLV","ADA-USD"],
    "P18": ["NFLX","BAC","JPM","JNJ","XOM","UNH","SPY","XLV","DBA","SOL-USD"],
    "P19": ["META","NVDA","TSLA","AAPL","MSFT","AMZN","QQQ","XLK","GLD","ETH-USD"],
    "P20": ["V","NKE","DIS","KO","ORCL","CSCO","IWM","XLE","USO","BTC-USD"],
    "P21": ["PFE","BAC","JPM","JNJ","UNH","XOM","EFA","XLF","DBC","BTC-USD"],
    "P22": ["AAPL","MSFT","NVDA","META","GOOGL","NFLX","SPY","VNQ","SLV","ETH-USD"],
    "P23": ["TSLA","AMZN","DIS","KO","ORCL","CSCO","QQQ","XLV","GLD","SOL-USD"],
    "P24": ["BAC","JPM","NKE","V","UNH","XOM","EEM","XLK","DBA","ADA-USD"],
    "P25": ["JNJ","PFE","META","NVDA","AMZN","AAPL","SPY","XLE","USO","BTC-USD"],
    "P26": ["MSFT","GOOGL","DIS","KO","ORCL","CSCO","IWM","EFA","GLD","ETH-USD"],
    "P27": ["NFLX","NKE","BAC","JPM","JNJ","V","VNQ","XLV","DBC","SOL-USD"],
    "P28": ["XOM","UNH","META","NVDA","TSLA","MSFT","QQQ","EEM","SLV","BTC-USD"],
    "P29": ["AMZN","AAPL","GOOGL","DIS","KO","ORCL","SPY","XLF","DBA","ADA-USD"],
    "P30": ["CSCO","PFE","BAC","JPM","JNJ","UNH","XLK","VNQ","USO","ETH-USD"],
}
len(PORTFOLIOS)


In [ ]:
# === EDIT THIS CELL: set your student_id and portfolio_id ===
student_id = "S01"   # <-- change to your student ID
portfolio_id = "P01" # <-- change to your assigned portfolio, e.g. "P07"

if portfolio_id not in PORTFOLIOS:
    raise ValueError(f"Unknown portfolio_id: {portfolio_id}")

tickers = PORTFOLIOS[portfolio_id]
weights = WEIGHTS_PATTERN

print("Student ID:", student_id)
print("Portfolio ID:", portfolio_id)
print("Tickers:", tickers)
print("Weights (should sum to 1):", weights.sum())


In [ ]:
# === Download asset prices and market index ===
prices = yf.download(tickers, start=START_DATE, end=END_DATE)["Adj Close"]
prices = prices.fillna(method="ffill").dropna(axis=1, how="any")
daily_returns = prices.pct_change().dropna()

mkt_prices = yf.download(MARKET_TICKER, start=START_DATE, end=END_DATE)["Adj Close"]
mkt_returns = mkt_prices.pct_change().dropna()

print("Asset price data shape:", prices.shape)
print("Market data length:", len(mkt_returns))


In [ ]:
# === Compute portfolio returns and value ===
# Ensure ordering is consistent
daily_returns = daily_returns[tickers]

# Portfolio daily returns (weighted sum)
port_rets = (daily_returns * weights).sum(axis=1)

# Portfolio value in USD and AED over time
growth = (1 + port_rets).cumprod()
portfolio_value_usd = BUDGET_USD * growth
portfolio_value_aed = portfolio_value_usd * AED_PER_USD

portfolio_value_aed.head()


In [ ]:
# === Run CAPM regression: rp - rf = alpha + beta * (rm - rf) + error ===
# Align portfolio and market returns
prt, mkt = port_rets.align(mkt_returns, join="inner")

prt_excess = prt - RF_DAILY
mkt_excess = mkt - RF_DAILY

X = sm.add_constant(mkt_excess)
capm_model = sm.OLS(prt_excess, X).fit()
display(capm_model.summary())

alpha_daily = float(capm_model.params.iloc[0])
beta = float(capm_model.params.iloc[1])
alpha_annual = alpha_daily * 252

print("Alpha (daily):", alpha_daily)
print("Alpha (annual):", alpha_annual)
print("Beta:", beta)


In [ ]:
# === Compute final P&L and total return ===
final_value_usd = float(portfolio_value_usd.iloc[-1])
final_value_aed = final_value_usd * AED_PER_USD

initial_invest_aed = BUDGET_AED
pnl_aed = final_value_aed - initial_invest_aed
total_return_pct = pnl_aed / initial_invest_aed

print(f"Initial Invested (AED): {initial_invest_aed:,.2f}")
print(f"Final Value     (AED): {final_value_aed:,.2f}")
print(f"PnL            (AED): {pnl_aed:,.2f}")
print(f"Total Return       : {100*total_return_pct:.2f}%")


In [ ]:
# === Optional: Sharpe ratio and other metrics ===
qs.extend_pandas()
sharpe = port_rets.sharpe(rf=RF_DAILY)
print("Daily Sharpe ratio:", sharpe)

qs.reports.metrics(port_rets, rf=RF_DAILY, display=True)


In [ ]:
# === Export one-row summary CSV for submission ===
summary = pd.DataFrame([{
    "student_id": student_id,
    "portfolio_id": portfolio_id,
    "alpha_daily": alpha_daily,
    "alpha_annual": alpha_annual,
    "beta": beta,
    "final_value_aed": final_value_aed,
    "pnl_aed": pnl_aed,
    "total_return_pct": total_return_pct,
    "sharpe_daily": float(sharpe),
}])

filename = f"summary_{student_id}_{portfolio_id}.csv"
summary.to_csv(filename, index=False)
print("Summary saved to:", filename)
summary
